# Building an AI agent with Strands Agents SDK and SAP GenAI Hub

This notebook demonstrates how to use the custom `SAPGenAIHubModel` class with the Strands Agents SDK to create an agent that uses SAP's GenAI Hub for consuming LLMs.

Note: If you have not set up your ai-core credentials yet, please follow notebook 00-load-sap-ai-core-credentials.

## Prerequisites

1. SAP AI Core credentials in your `~/.aicore/config.json` file
2. Strands Agents SDK installed
3. SAP GenAI Hub SDK installed

## Install Dependencies

In [ ]:
%pip install --upgrade pip
%pip install .

## Import the SAPGenAIHubModel class

First, let's import the custom `SAPGenAIHubModel` class that we created to work with SAP's GenAI Hub.

In [ ]:
import os
import sys

from util.strands_bedrock_sap_genai_hub import SAPGenAIHubModel
from strands import Agent, tool


## Initialize the SAPGenAIHubModel

Now, let's initialize the `SAPGenAIHubModel` with the desired model from SAP's GenAI Hub. We'll use the Amazon Nova Lite model as an example.

In [ ]:
# Set the model ID for the SAP GenAI Hub model

# Available models include:
# from https://me.sap.com/notes/0003437766, 
# ensure that you have run the pre-requisites of the workshop https://catalog.us-east-1.prod.workshops.aws/workshops/6c1a3147-7c51-4b64-a3ad-5c302b7b41d8/en-US/20-setup-sap-genai-hub-and-amazon-bedrock/1-index.
# 
# - Amazon Nova models: "amazon--nova-pro", "amazon--nova-lite", "amazon--nova-micro"
# - Anthropic Claude models: "anthropic--claude-3.5-sonnet", "anthropic--claude-3-opus", "anthropic--claude-3-sonnet", "anthropic--claude-3-haiku"
# - Amazon Titan Text models: "amazon--titan-text-lite", "amazon--titan-text-express"
# - Amazon Titan Embedding models: "amazon--titan-embed-text"

model = SAPGenAIHubModel(model_id="amazon--nova-pro",
                        #  temperature = 0.3,
                        #  top_p = 1,
                        #  max_tokens = 25, 
                        #  stop_sequences = [ "blab" ],
                         )

## Create a Simple Agent

Let's create a simple agent using the `SAPGenAIHubModel` and test it with a basic query.

In [ ]:
# Create an agent with the SAP GenAI Hub model
agent = Agent(model=model)

# Test the agent with a simple query
# response = agent("What is the capital of Germany?")
response = agent("What is the name of the 32nd US president?")
print(response.message)

## Create an Agent with System Prompt

Now, let's create an agent with a custom system prompt to guide the model's behavior.

In [ ]:
# Create an agent with a system prompt
agent_with_system_prompt = Agent(
    model=model,
    system_prompt="You are a helpful assistant that specializes in geography and only reply in german. Always provide detailed information about locations."
)

# Test the agent with a geography question
response = agent_with_system_prompt("Tell me about Berlin.")
print(response.message)

## Create a Custom Tool

Let's create a custom tool for web search and use it with our agent.

In [ ]:
# Install duckduckgo-search if not already installed
%pip install . 

In [ ]:
# from duckduckgo_search import DDGS
from ddgs import DDGS

# Define custom tool
@tool
def web_search(query: str, max_results=5) -> dict:
    """Search the web for information."""
    results = ""
    with DDGS() as ddgs:
        results = [r for r in ddgs.text(query, max_results=max_results)]
    return results

# Create an agent with the custom tool
agent_with_custom_tool = Agent(
    model=model,
    tools=[web_search],
    system_prompt="You are a helpful assistant that can search the web for information."
)

# Test the agent with a search query
response = agent_with_custom_tool("What are the latest developments in quantum computing?")
print(response.message)

In [ ]:
agent_with_custom_tool.messages

## Using Different Models

Let's try using different models from SAP's GenAI Hub.

### Using Anthropic Claude Model

In [ ]:
# Initialize the SAPGenAIHubModel with Claude 3 Sonnet
claude_model = SAPGenAIHubModel(model_id="anthropic--claude-3.5-sonnet",
                                #  temperature = 0.3,
                                #  top_p = 1,
                                #  max_tokens = 25, 
                                #  stop_sequences = [ "blab" ],
                                )

# Create an agent with the Claude model
claude_agent = Agent(
    model=claude_model,
    system_prompt="You are a helpful assistant that specializes in creative writing."
)

# Test the agent with a creative writing task
response = claude_agent("Write a short poem about artificial intelligence.")
print(response.message)

## Multi-Agent Workflow

Finally, let's create a multi-agent workflow using the "Agents as Tools" pattern with our custom `SAPGenAIHubModel`.

In [ ]:
from textwrap import dedent

@tool
def research_assistant(query: str) -> str:
    """Research assistant that can search for information."""
    research_agent = Agent(
        model=claude_model,
        system_prompt=dedent(
            """You are a specialized research assistant. Focus only on providing
            factual, well-sourced information in response to research questions.
            Always cite your sources when possible."""
        ),
        tools=[web_search]
    )
    return research_agent(query).message

@tool
def creative_writing_assistant(query: str) -> str:
    """Creative writing assistant that can generate creative content."""
    creative_agent = Agent(
        model=model,
        system_prompt=dedent(
            """You are a specialized creative writing assistant.
            Create engaging and imaginative content based on user requests."""
        )
    )
    return creative_agent(query).message

# Define orchestrator system prompt
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For creative writing and content generation → Use the creative_writing_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

# Create the orchestrator agent
orchestrator = Agent(
    model=model,
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[research_assistant, creative_writing_assistant]
)

In [ ]:
# Test the orchestrator with a research question
response = orchestrator("What is the history of artificial intelligence?")
print(response.message)

In [ ]:
# to understand what is happening between the orchestrator and sub-agents, you can uncomment list the messages
# orchestrator.messages

In [ ]:
# Test the orchestrator with a creative writing request
response = orchestrator("Write a short story about a robot learning to be human.")
print(response.message)

## Conclusion

In this notebook, we demonstrated how to use the custom `SAPGenAIHubModel` class with the Strands Agents SDK to create AI agents that leverage SAP's GenAI Hub for consuming LLMs. We showed how to:

1. Initialize the `SAPGenAIHubModel` with different models from SAP's GenAI Hub
2. Create simple agents with and without system prompts
3. Enhance agents with built-in and custom tools
4. Create multi-agent workflows using the "Agents as Tools" pattern

This integration allows you to leverage the power of SAP's GenAI Hub with the flexibility and extensibility of the Strands Agents SDK.

In [ ]:
# In case you want to troubleshoot something, the following debugs will help

# import logging

# logging.basicConfig(level=logging.DEBUG)
# logger = logging.getLogger()
# logger.setLevel(logging.DEBUG)